In [2]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint
import fn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# make page
paper_size = '14x11 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
lines = []

node_sets = []
n_lines = 500
n_nodes_per_line = 40
y_start = 0
y_end = 14
x_start = 0
x_end = 10
node_x_centers = np.linspace(x_start, x_end, n_lines)
std_scale = 0.09
n_eval_points = 80

### initialize vals
node_ys = np.linspace(y_start, y_end, n_nodes_per_line)
centered_node_xs = np.zeros(node_ys.shape)
bez_eval_end_center = 1
bez_eval_end_noise = 0
bez_eval_end_limit = 1.
bez_eval_end_std_scale = 0.01

for i, node_x_center in enumerate(node_x_centers):
    new_x_noise = np.random.randn(n_nodes_per_line) * std_scale
    centered_node_xs = centered_node_xs + new_x_noise
    node_xs = node_x_center + centered_node_xs
    node_xs[:3] = node_x_center
    node_xs[-3:] = node_x_center
    nodes = np.asfortranarray([
        node_xs,
        node_ys,
        ])
    curve = bezier.Curve(nodes, degree=(nodes.shape[1]-1))
    eval_start = np.random.uniform(0, 0.03)
    eval_end = np.random.uniform(0.97, 1.)
    eval_points = np.linspace(eval_start, eval_end, n_eval_points)
    x, y = curve.evaluate_multi(eval_points)
    if i % 2:
        x = np.flipud(x)
        y = np.flipud(y)
    lines.append(np.stack([x, y]).T)
    node_sets.append(np.stack([node_xs, node_ys]).T)

In [ ]:
ls = [LineString(l) for l in lines]

mls = gp.make_like(gp.merge_LineStrings(ls), drawbox)

mask = drawbox

in_mask = mls.intersection(mask)

in_mask = sa.rotate(in_mask, -90)

split_point = 500

layer1 = in_mask[:split_point]
layer2 = in_mask[split_point:]

layers = []
layers.append(LineString(np.concatenate([np.array(l) for l in layer1])))
layers.append(LineString(np.concatenate([np.array(l) for l in layer2])))

# layers = [in_mask]

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesort')

sk.display(color_mode='none')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/246_500_lines.svg')

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=55
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
def oscillator(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v) + (a * v) + (b * u)
    dudt = np.cos(u) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

def oscillator2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v) + np.sin(u) + (a * v) + (b * u)
    dudt = np.cos(u) + np.cos(u) ** 2 + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(v)  + np.cos(u * v) + (a * v) + (b * u)
    dudt = np.cos(u) + np.sin(u * v) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 500
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(.5, 4.5, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius
    y = np.sin(theta) * radius
    pts.append(Point(x, y))

In [ ]:
lfs = []


t_max = 3.7
t = np.linspace(0, t_max, 1801)
a = -0.4
b = 0.3
c = 0.75
d = -0.2



for pt in tqdm(pts):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lines)

sk.penWidth('0.3')
sk.vpype('linesimplify linesort ')

sk.display(color_mode='none')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/247_500_lines.svg')

# Try 2

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=55
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 500
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(.75, 3.45, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [ ]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + (c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 1801)
a = -0.2
b = -0.2
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
layers = []
layers.append(lines[:250])
layers.append(lines[250:])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/249_500_lines.svg')

# Try 3

In [ ]:
# make page
paper_size = '17x23.5 inches'
border:float=35
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 3500
thetas = np.linspace(0, np.pi*14, n_lines)
radii = np.linspace(.5, 5.45, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [ ]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + np.cos(c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [ ]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 701)
a = -0.2
b = -0.25
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

In [ ]:
lbs = lines.buffer(0.07, cap_style=2, join_style=2).boundary

In [ ]:
lbs = gp.merge_LineStrings([l for l in lbs if l.length > 0.9])

In [ ]:
n_layers = 1
layer_inds = np.split(np.arange(len(lbs)), n_layers)

In [ ]:
layers = []
for ind_set in layer_inds:
    layer = [lbs[i] for i in ind_set]
    layers.append(gp.merge_LineStrings(layer))
    

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk.save('/mnt/c/code/side/plotter_images/oned_outputs/251_3500_lines.svg')

# Try 4

In [3]:
# make page
paper_size = '11x14 inches'
border:float=25
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [4]:
center = drawbox.centroid

In [5]:
n_lines = 500
thetas = np.linspace(0, np.pi*14, n_lines)
radii = np.linspace(.5, 5.45, n_lines)

In [6]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 3.3
    y = np.sin(theta) * radius + 0.5
    pts.append(Point(x, y))

In [7]:
def ode2(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(u * v + (a * v) + (b * u))
    dudt = np.cos(u) + np.sin(u * v) + np.cos(c * v) + (d * u)
    dydt = [dvdt, dudt]
    return dydt

In [8]:
lfs = []


t_max = 2.7
t = np.linspace(0, t_max, 701)
a = -0.2
b = -0.25
c = 0.04
d = -0.25



for pt in tqdm(pts):
    sol = odeint(ode2, [pt.x, pt.y], t, args=(a, b, c, d))
    lfs.append(LineString(sol))
    

lines = gp.make_like(MultiLineString(lfs), drawbox)

100%|██████████| 500/500 [00:00<00:00, 784.67it/s]


In [9]:
lbs = lines.buffer(0.07, cap_style=2, join_style=2).boundary

In [10]:
lbs = gp.merge_LineStrings([l for l in lbs if l.length > 0.9])

In [11]:
n_layers = 1
layer_inds = np.split(np.arange(len(lbs)), n_layers)

In [12]:
layers = []
for ind_set in layer_inds:
    layer = [lbs[i] for i in ind_set]
    layers.append(gp.merge_LineStrings(layer))
    

In [13]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
sk.vpype('linesimplify')

sk.display(color_mode='layer')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-10-05T23:22:16.608680 
 
 
 
 
 
 <polygon points="910.5302,597.3449 909.5121,605.5514 909.2674,613.7386 909.7452,622.4437 910.9698,631.5368 912.9209,640.8175 915.7606,650.7737 919.1744,660.2528 923.2083,669.5191 928.1535,679.048 933.7275,688.0539 939.6829,696.0341 945.3517,702.1206 949.6974,705.5571 953.3217,707.1102 956.394,707.0495 961.6695,705.3587 961.4957,704.859 957.6647,706.077 955.3897,706.2579 953.2385,705.6607 950.819,704.2625 947.3818,701.4287 943.6115,697.5344 939.7159,692.8096 935.7295,687.2718 928.48,675.1852 922.3731,662.3087 917.8485,650.0787 915.4172,641.6809 913.5206,633.1839 913.001,633.2838 915.7352,644.9015 919.659,656.8938 924.4976,668.4061 930.0161,679.065 936.4783,689.2931 943.3709,698.0596 946.6864,701.4999 949.7849,704.1643 952.4572,705.8759 954.823,706.7116 951.945,706.1381 948.6354,704.1891 944.6404,700.7041 940.2698,695.9123 935.813,690.1249 931.6265,683.8531 927.4742,676.7352 923.6863,669.2922 919.9447,660.7537 916.5082,651.3593 913.8243,642.2321 911.7662,632.9805 910.5069,624.6267 909.8464,615.8523 909.9933,606.2473 911.1592,596.8381 910.6384,596.7443"/>
 <polygon points="904.8618,560.959 899.1269,569.4494 894.6739,578.3637 891.5073,587.5088 889.3272,597.7997 888.3289,608.5787 888.4888,620.3045 889.7722,632.0201 892.1587,644.0822 896.2798,658.232 901.4104,671.3337 907.8381,684.156 914.9915,695.3901 918.5935,700.0688 922.333,704.2723 925.9492,707.6814 929.2679,710.1841 932.4127,711.9162 935.4205,712.8929 938.3657,713.1401 941.5214,712.6829 948.9892,710.0909 948.7971,709.5978 942.7045,711.8123 938.8239,712.5859 935.5966,712.3886 932.3444,711.3048 928.2904,708.8608 923.9907,705.1844 919.4456,700.266 914.9104,694.3672 908.0089,683.3798 901.8942,671.1193 896.7805,658.0607 892.6731,643.9582 890.2951,631.9395 889.0168,620.2709 888.8576,608.5992 889.8505,597.878 891.8694,588.1884 894.7698,579.5229 899.343,570.1307 902.3433,565.3447 905.5562,560.9256 905.1374,560.6021"/>
 <polygon points="771.8891,878.3979 776.5298,882.391 782.4779,885.7662 788.4982,887.8927 794.6257,888.8037 800.3692,888.4302 805.6938,886.8743 810.5687,884.2415 815.3539,880.2963 818.5112,876.8125 821.4143,872.8954 824.2784,868.2916 827.2643,862.6635 832.854,849.9405 846.4861,813.6345 853.7851,796.2782 858.1555,787.2554 862.7684,778.8001 867.4611,771.2059 872.45,764.0942 879.7944,754.9844 884.8076,749.7364 892.2604,742.8569 900.3839,736.3697 908.9798,730.3958 918.4388,724.6455 929.4535,718.794 939.8788,713.9261 939.6661,713.4417 925.2521,719.9999 919.7219,722.1232 914.9204,723.1602 910.4352,723.0942 907.7394,722.5114 904.9401,721.4756 899.4243,718.2174 893.6944,713.2433 888.0206,706.7875 881.4763,697.3552 875.6502,686.8427 870.4314,675.132 865.9099,662.2364 862.5226,649.4476 860.1347,636.5632 858.7349,623.2591 858.4579,610.5641 857.9288,610.554 857.8966,613.1731 858.4072,626.0609 859.6111,636.6397 861.385,646.6857 863.5851,656.016 866.5116,665.8782 869.9396,675.3275 873.7507,684.1315 878.087,692.614 882.4886,699.9402 887.3288,706.7614 892.3338,712.6046 897.2269,717.1825 902.1691,720.6474 907.0183,722.843 911.7016,723.7559 908.1223,723.3614 904.4187,722.2011 900.6897,720.2875 896.9299,717.6448 893.0913,714.2444 889.0637,709.932 885.2464,705.1304 881.5232,699.7375 874.8196,687.9815 868.8056,674.4362 864.3762,661.4991 861.9022,652.2932 859.8081,642.2807 859.2874,642.3749 862.0984,655.2749 865.8633,667.9014 870.4212,679.7888 875.7975,691.0164 880.7864,699.5695 886.2198,707.2875 891.743,713.6454 897.4481,718.7153 902.7336,722.0176 907.8791,723.8505 912.9853,724.2556 918.3705,723.2746 913.6564,724.8559 909.4886,725.5489 905.5645,725.4178 901.5628,724.4379 897.4588,722.5598 893.2315,719.7484 889.0777,716.1603 884.7076,711.5194 880.26,705.8624 876.2004,699.8121 872.1122,692.7433 868.4185,685.3424 864.9466,677.26 862.0309,669.3288 859.4609,661.0819 857.1192,651.9414 855.1485,642.0239 853.7635,632.185 852.9599,622.5551 852.7254,612.5861 853.1119,603.0729 854.0872,594.0859 855.

In [113]:
plot_id = fn.new_plot_id()

saved 20210922-224529_191352-4e22b-c23cd2 to s3://algorithmic-ink/current_plot_id


In [114]:
savedir='/home/naka/art/plotter_svgs'

In [115]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)